In [27]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [43]:
def weight_variable(shape,astype):
    initial = tf.truncated_normal(shape, stddev=0.01, dtype = astype)
    return tf.Variable(initial, dtype = astype)


def bias_variable(shape,astype):
    initial = tf.constant(0.0, shape=shape, dtype = astype)
    return tf.Variable(initial, dtype = astype)

def read_input():
    inp_file = open(r'C:\Users\anant\dev\repos\HPA\inputs', 'rb')
    inp = pickle.load(inp_file)
    inp_file.close()
    
    out_file = open(r'C:\Users\anant\dev\repos\HPA\outputs', 'rb')
    out = pickle.load(out_file)
    out_file.close()

    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoded = onehot_encoder.fit_transform(out.reshape(len(out), 1))

    inp_train, inp_val, onehot_encoded_train, onehot_encoded_val = train_test_split(inp, onehot_encoded)
    print(inp_train.shape, onehot_encoded_train.shape, inp_val.shape, onehot_encoded_val.shape)
    return inp_train, onehot_encoded_train, inp_val, onehot_encoded_val

def nn_example():
    inp, out, inp_va, out_va = read_input()
    no_of_data = inp.shape[0]
    input_feature_count = inp.shape[1]
    out_classes = out.shape[1]
    print('Features={}, classes={}'.format(input_feature_count, out_classes))
    print('Inp_train={}, inp_val={}, out_train={}, out_val={}'.format(inp.shape, inp_va.shape, out.shape, out_va.shape))
    
    # Python optimisation variables
    learning_rate = 0.5
    epochs = 10
    batch_size = 100
    
    # Neural network hidden layer variables
    h1 = 4
    h2 = 3

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(tf.float32, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(tf.float32, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(tf.float32)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], tf.float32)
    b_fc1 = bias_variable([h1], tf.float32)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],tf.float32)
    b_fc2 = bias_variable([h2], tf.float32)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],tf.float32)
    b_fc3 = bias_variable([out_classes], tf.float32)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)

    merged = tf.summary.merge([accuracy_sum])
    writer = tf.summary.FileWriter(r'C:\Users\anant\dev\repos\HPA')
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = no_of_data//batch_size
        print('Total number of batches to train={}'.format(total_batch))
        for epoch in range(epochs):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = inp[i*batch_size:(i+1)*batch_size], out[i*batch_size:(i+1)*batch_size]
                train_acc, _, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
            print("Epoch:", (epoch + 1), "Train accuracy =", "{:.3f}".format(train_acc), "cost =", "{:.3f}".format(avg_cost))
#             summary = sess.run(merged, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
#             writer.add_summary(summary, epoch)

        print("\nTraining complete!")
#         writer.add_graph(sess.graph)
        print(sess.run(accuracy, feed_dict={x: inp_va, y: out_va, keep_prob_input: 1.0, keep_prob: 1.0}))

if __name__ == "__main__":
    nn_example()

(7500, 8) (7500, 2) (2500, 8) (2500, 2)
Features=8, classes=2
Inp_train=(7500, 8), inp_val=(2500, 8), out_train=(7500, 2), out_val=(2500, 2)
Total number of batches to train=75
Epoch: 1 Train accuracy = 0.640 cost = 1.325
Epoch: 2 Train accuracy = 0.640 cost = 1.324
Epoch: 3 Train accuracy = 0.640 cost = 1.324
Epoch: 4 Train accuracy = 0.640 cost = 1.324
Epoch: 5 Train accuracy = 0.640 cost = 1.324
Epoch: 6 Train accuracy = 0.640 cost = 1.324
Epoch: 7 Train accuracy = 0.640 cost = 1.324
Epoch: 8 Train accuracy = 0.640 cost = 1.324
Epoch: 9 Train accuracy = 0.640 cost = 1.324
Epoch: 10 Train accuracy = 0.640 cost = 1.324

Training complete!
0.6336


## Below cell commented out

In [47]:
def nn_example():
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

    # Python optimisation variables
    learning_rate = 0.5
    epochs = 10
    batch_size = 100

    # Python optimisation variables
    learning_rate = 0.5
    epochs = 10
    batch_size = 100
    input_feature_count = 784
    out_classes = 10
    
    # Neural network hidden layer variables
    h1 = 4
    h2 = 3

    # declare the training data placeholders
    # input x - for 28 x 28 pixels = 784
    x = tf.placeholder(tf.float32, [None, input_feature_count])
    # now declare the output data placeholder - 10 digits
    y = tf.placeholder(tf.float32, [None, out_classes])

    # build the network
    keep_prob_input = tf.placeholder(tf.float32)
    x_drop = tf.nn.dropout(x, keep_prob=keep_prob_input)

    W_fc1 = weight_variable([input_feature_count, h1], tf.float32)
    b_fc1 = bias_variable([h1], tf.float32)
    h_fc1 = tf.nn.relu(tf.matmul(x_drop, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([h1, h2],tf.float32)
    b_fc2 = bias_variable([h2], tf.float32)
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

    W_fc3 = weight_variable([h2, out_classes],tf.float32)
    b_fc3 = bias_variable([out_classes], tf.float32)

    # now calculate the hidden layer output - in this case, let's use a softmax activated
    # output layer
    y_ = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

    # now let's define the cost function which we are going to train the model on
    y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
    cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                                                  + (1 - y) * tf.log(1 - y_clipped), axis=1))

    # add an optimiser
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

    # finally setup the initialisation operator
    init_op = tf.global_variables_initializer()

    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # add a summary to store the accuracy
    accuracy_sum = tf.summary.scalar('accuracy', accuracy)

    merged = tf.summary.merge([accuracy_sum])
    writer = tf.summary.FileWriter(r'C:\Users\anant\dev\repos\HPA')
    # start the session
    with tf.Session() as sess:
        # initialise the variables
        sess.run(init_op)
        total_batch = int(len(mnist.train.labels) / batch_size)
        for epoch in range(epochs):
            avg_cost = 0
            train_acc = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                train_acc,_, c = sess.run([accuracy, optimiser, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob_input: 1.0, keep_prob: 1.0})
                avg_cost += c / total_batch
            print("Epoch:", (epoch + 1), "Train accuracy =", "{:.3f}".format(train_acc), "cost =", "{:.3f}".format(avg_cost))
            summary = sess.run(merged, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0})
            writer.add_summary(summary, epoch)

        print("\nTraining complete!")
        writer.add_graph(sess.graph)
        print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob_input: 1.0, keep_prob: 1.0}))

if __name__ == "__main__":
    nn_example()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 1 Train accuracy = 0.360 cost = 2.964
Epoch: 2 Train accuracy = 0.100 cost = 3.191
Epoch: 3 Train accuracy = 0.090 cost = 3.251
Epoch: 4 Train accuracy = 0.140 cost = 3.251
Epoch: 5 Train accuracy = 0.120 cost = 3.251
Epoch: 6 Train accuracy = 0.080 cost = 3.251
Epoch: 7 Train accuracy = 0.150 cost = 3.251
Epoch: 8 Train accuracy = 0.150 cost = 3.251
Epoch: 9 Train accuracy = 0.080 cost = 3.251
Epoch: 10 Train accuracy = 0.090 cost = 3.251

Training complete!
0.1135
